In [2]:
library(bnlearn)
library(Rgraphviz)

Today i learnt that the k2 algorithm infers the dag, the directed acyclic graph, so the structure of the bayesian network from the dataset. Not from the fitted model. 
What i initially thought was that you happen to have a model and you want to reverse engineer that. 

In [6]:
# Trying implementing k2 using learing.test dataset
head(learning.test)
colnames(learning.test)

,A,B,C,D,E,F
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,b,c,b,a,b,b
2,b,a,c,a,b,b
3,a,a,a,a,a,a
4,a,a,a,a,b,b
5,a,a,b,c,a,a
6,c,c,a,c,c,a


[1] "A" "B" "C" "D" "E" "F"

In [170]:
node.values <- function(dataset=dataset, parents) {
    # retrieves the unique values of the nodes
    #return a list of vectors, 
    # each vector contains the unique values of the corresponding node
    output <- list()
    for (i in seq_along(parents)) {
        node <- parents[i]
        node.values <- levels(unique(dataset[[node]]))
        output[[i]] <- node.values
    }
    return(output)
}

alphaijk <- function(i=i, pii=pii, dataset=dataset, nodes.order=nodes.order) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    cases <- expand.grid(node.values(parents=all.nodes))
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(parents=all.nodes))
    output <- c()
    for (i in 1:dim(unique.instantiations)[1]){
        aa <- 0
        for (j in 1:dim(subset)[1]) {
            ifelse(all(subset[j,] == unique.instantiations[i,]), aa <- aa +1, FALSE)
        }
        output <- c(output,aa)
    }
    return(output)
}

k2helper <- function(i=i, pii=pii, dataset=dataset) {
    # the "f" function in the pdf
    phi_i <- expand.grid(node.values(dataset, pii))
    q_i <- dim(phi_i)[1]
    nodes <- colnames(dataset)
    i.th.node <- nodes[i]
    v_i <- node.values(dataset, i.th.node)
    r_i <- length(v_i[[1]])
    aijk <- prod(factorial(alphaijk()))
    numeratore <- factorial(r_i-1)
    nij <- alphaijk(i=0) # i = 0 deactivates actual node
    nij <- nij + r_i-1
    denominatore <- factorial(nij)
    frazione <- numeratore/denominatore
    return(prod(frazione)* aijk)
}




k2 <- function(dataset, nodes.order, upperbound) {
    nodes <- colnames(dataset)
    for (i in seq_along(nodes.order)) {
        actual_node <- nodes.order[i]
        pii <- c()
        p.old <- k2helper(i, pii)
        proceed <- TRUE
        while (proceed & pii < upperbound) {
            p.new <- k2helper(i, pii + z)
            if (p.new > p.old) {
                p.old <- p.new
                pii <- pii + 1
            } else {
                proceed <- FALSE
            }
        } 
    }
}

In [169]:
2/c(1,2,3,4) 

[1] 2.0000000 1.0000000 0.6666667 0.5000000